In [1]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('training.psv', delimiter = '|')
eval_data = pd.read_csv('eval.psv', delimiter = '|')
train_data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97276 entries, 0 to 97275
Data columns (total 5 columns):
student_id    97276 non-null object
level         97276 non-null object
course        97276 non-null object
grade         97276 non-null object
major         97276 non-null object
dtypes: object(5)
memory usage: 3.7+ MB


In [2]:
eval_data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19440 entries, 0 to 19439
Data columns (total 7 columns):
student_id    19440 non-null object
level         19440 non-null object
course        19440 non-null object
grade         19440 non-null object
major1        0 non-null float64
major2        0 non-null float64
major3        0 non-null float64
dtypes: float64(3), object(4)
memory usage: 1.0+ MB


In [3]:
train_data.head()

,student_id,level,course,grade,major
0,ppVGBRKhtqqyxnVO,Freshman,SPAN:100,A,Business
1,PiPkSgMGbFIu5RwR,Freshman,CSI:160,S,International Relations
2,PiPkSgMGbFIu5RwR,Sophomore,EES:107,C,International Relations
3,PiPkSgMGbFIu5RwR,Senior,SPAN:201,B,International Relations
4,PiPkSgMGbFIu5RwR,Junior,ENTR:200,B+,International Relations


In [4]:
eval_data.head()

,student_id,level,course,grade,major1,major2,major3
0,6Q79FVMSbTPMBhha,Senior,ECE:489,A,NaN,NaN,NaN
1,6Q79FVMSbTPMBhha,Sophomore,MATH:246,B,NaN,NaN,NaN
2,6Q79FVMSbTPMBhha,Sophomore,ECE:344,C+,NaN,NaN,NaN
3,6Q79FVMSbTPMBhha,Junior,ARTS:280,B,NaN,NaN,NaN
4,6Q79FVMSbTPMBhha,Freshman,CSI:160,AUS,NaN,NaN,NaN


In [5]:
train_data.shape

(97276, 5)

In [6]:
eval_data.shape

(19440, 7)

In [7]:
#summary statistics
train_data.describe()

,student_id,level,course,grade,major
count,97276,97276,97276,97276,97276
unique,10000,4,2721,21,81
top,cJdq8OlOg2x4drwW,Senior,RHET:103,A,Psychology
freq,90,30367,1685,18729,10274


In [8]:
#summary statistics
eval_data.describe()

,major1,major2,major3
count,0.0,0.0,0.0
mean,NaN,NaN,NaN
std,NaN,NaN,NaN
min,NaN,NaN,NaN
25%,NaN,NaN,NaN
50%,NaN,NaN,NaN
75%,NaN,NaN,NaN
max,NaN,NaN,NaN


In [9]:
#double check whether there are any empty cells in the data
train_data.isna().any()

student_id    False
level         False
course        False
grade         False
major         False
dtype: bool

In [10]:
#double check whether there are any empty cells in the data
eval_data.isna().any()

student_id    False
level         False
course        False
grade         False
major1         True
major2         True
major3         True
dtype: bool

In [11]:
split = train_data.shape[0]
X_train = train_data.iloc[:, [0,1,2,3]]
X_eval = eval_data.iloc[:, [0,1,2,3]]
dfs = [X_train, X_eval]
data = pd.concat(dfs, ignore_index = True)
data.shape

(116716, 4)

In [12]:
from sklearn.preprocessing import LabelEncoder
'''labelencoder_0 = LabelEncoder()
X_train[:,0] = labelencoder_0.fit_transform(X_train[:,0])
labelencoder_1 = LabelEncoder()
X_train[:,1] = labelencoder_1.fit_transform(X_train[:,1])
labelencoder_2 = LabelEncoder()
X_train[:,2] = labelencoder_2.fit_transform(X_train[:,2])
labelencoder_3 = LabelEncoder()
X_train[:,3] = labelencoder_3.fit_transform(X_train[:,3])
labelencoder_y = LabelEncoder()
y_train = labelencoder_y.fit_transform(y_train)'''
labelencoder = LabelEncoder()
data =data.apply(labelencoder.fit_transform)
y_train = labelencoder.fit_transform(y_train)

In [13]:
X_train = data.iloc[:split,:].values
X_train.shape

In [15]:
X_eval = data.iloc[split:,:].values
X_eval.shape

(19440, 4)

In [19]:
y_train = train_data.iloc[:, 4].values
y_train.shape

(97276,)

In [20]:
#splitting the X_train and y_train data into train and test set for validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train, test_size=0.30, random_state=5)

In [21]:
###parameter tuning
from xgboost import XGBClassifier

# validate model on training data
model = XGBClassifier(n_estimators=9000, objective= 'multi:softmax', num_class=81, learning_rate = 0.4, reg_lambda = .1, 
                      max_depth =3, colsample_bytree = 0.2, subsample =1, min_child_weight =2, tree_method = "auto", 
                      nthread = -1, random_state = 41)
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_metric = 'mlogloss', eval_set=eval_set, early_stopping_rounds = 10, verbose=True)

[0]	validation_0-mlogloss:3.84792
Will train until validation_0-mlogloss hasn't improved in 10 rounds.
[1]	validation_0-mlogloss:3.49177
[2]	validation_0-mlogloss:3.22864
[3]	validation_0-mlogloss:3.06227
[4]	validation_0-mlogloss:2.93181
[5]	validation_0-mlogloss:2.83821
[6]	validation_0-mlogloss:2.66393
[7]	validation_0-mlogloss:2.58993
[8]	validation_0-mlogloss:2.49901
[9]	validation_0-mlogloss:2.46201
[10]	validation_0-mlogloss:2.44607
[11]	validation_0-mlogloss:2.41979
[12]	validation_0-mlogloss:2.3859
[13]	validation_0-mlogloss:2.35518
[14]	validation_0-mlogloss:2.32411
[15]	validation_0-mlogloss:2.35721
[16]	validation_0-mlogloss:2.3169
[17]	validation_0-mlogloss:2.26016
[18]	validation_0-mlogloss:2.22227
[19]	validation_0-mlogloss:2.23575
[20]	validation_0-mlogloss:2.20349
[21]	validation_0-mlogloss:2.18183
[22]	validation_0-mlogloss:2.16405
[23]	validation_0-mlogloss:2.13859
[24]	validation_0-mlogloss:2.10384
[25]	validation_0-mlogloss:2.09125
[26]	validation_0-mlogloss:2.0726

[230]	validation_0-mlogloss:1.10171
[231]	validation_0-mlogloss:1.09814
[232]	validation_0-mlogloss:1.09735
[233]	validation_0-mlogloss:1.09504
[234]	validation_0-mlogloss:1.09268
[235]	validation_0-mlogloss:1.09069
[236]	validation_0-mlogloss:1.08702
[237]	validation_0-mlogloss:1.08371
[238]	validation_0-mlogloss:1.08081
[239]	validation_0-mlogloss:1.07922
[240]	validation_0-mlogloss:1.0773
[241]	validation_0-mlogloss:1.07589
[242]	validation_0-mlogloss:1.07469
[243]	validation_0-mlogloss:1.07154
[244]	validation_0-mlogloss:1.06872
[245]	validation_0-mlogloss:1.06733
[246]	validation_0-mlogloss:1.06577
[247]	validation_0-mlogloss:1.06354
[248]	validation_0-mlogloss:1.06147
[249]	validation_0-mlogloss:1.05939
[250]	validation_0-mlogloss:1.05742
[251]	validation_0-mlogloss:1.05656
[252]	validation_0-mlogloss:1.05485
[253]	validation_0-mlogloss:1.05325
[254]	validation_0-mlogloss:1.05066
[255]	validation_0-mlogloss:1.04893
[256]	validation_0-mlogloss:1.04746
[257]	validation_0-mlogloss:1

[454]	validation_0-mlogloss:0.766806
[455]	validation_0-mlogloss:0.765885
[456]	validation_0-mlogloss:0.764262
[457]	validation_0-mlogloss:0.762763
[458]	validation_0-mlogloss:0.761636
[459]	validation_0-mlogloss:0.758937
[460]	validation_0-mlogloss:0.758157
[461]	validation_0-mlogloss:0.757603
[462]	validation_0-mlogloss:0.756614
[463]	validation_0-mlogloss:0.756136
[464]	validation_0-mlogloss:0.75527
[465]	validation_0-mlogloss:0.754396
[466]	validation_0-mlogloss:0.753592
[467]	validation_0-mlogloss:0.752815
[468]	validation_0-mlogloss:0.750972
[469]	validation_0-mlogloss:0.750318
[470]	validation_0-mlogloss:0.748526
[471]	validation_0-mlogloss:0.747845
[472]	validation_0-mlogloss:0.747026
[473]	validation_0-mlogloss:0.746781
[474]	validation_0-mlogloss:0.745615
[475]	validation_0-mlogloss:0.743975
[476]	validation_0-mlogloss:0.743011
[477]	validation_0-mlogloss:0.742405
[478]	validation_0-mlogloss:0.741083
[479]	validation_0-mlogloss:0.739617
[480]	validation_0-mlogloss:0.7392
[481

[677]	validation_0-mlogloss:0.577593
[678]	validation_0-mlogloss:0.577045
[679]	validation_0-mlogloss:0.576454
[680]	validation_0-mlogloss:0.575726
[681]	validation_0-mlogloss:0.574861
[682]	validation_0-mlogloss:0.574101
[683]	validation_0-mlogloss:0.573707
[684]	validation_0-mlogloss:0.572947
[685]	validation_0-mlogloss:0.572492
[686]	validation_0-mlogloss:0.571992
[687]	validation_0-mlogloss:0.571364
[688]	validation_0-mlogloss:0.571143
[689]	validation_0-mlogloss:0.569813
[690]	validation_0-mlogloss:0.568942
[691]	validation_0-mlogloss:0.568503
[692]	validation_0-mlogloss:0.567799
[693]	validation_0-mlogloss:0.567043
[694]	validation_0-mlogloss:0.566712
[695]	validation_0-mlogloss:0.566101
[696]	validation_0-mlogloss:0.565488
[697]	validation_0-mlogloss:0.564579
[698]	validation_0-mlogloss:0.564171
[699]	validation_0-mlogloss:0.563313
[700]	validation_0-mlogloss:0.562556
[701]	validation_0-mlogloss:0.561877
[702]	validation_0-mlogloss:0.560902
[703]	validation_0-mlogloss:0.560186
[

[900]	validation_0-mlogloss:0.459106
[901]	validation_0-mlogloss:0.458693
[902]	validation_0-mlogloss:0.458487
[903]	validation_0-mlogloss:0.458324
[904]	validation_0-mlogloss:0.458119
[905]	validation_0-mlogloss:0.457598
[906]	validation_0-mlogloss:0.457317
[907]	validation_0-mlogloss:0.456782
[908]	validation_0-mlogloss:0.45619
[909]	validation_0-mlogloss:0.455957
[910]	validation_0-mlogloss:0.455142
[911]	validation_0-mlogloss:0.45484
[912]	validation_0-mlogloss:0.454525
[913]	validation_0-mlogloss:0.45404
[914]	validation_0-mlogloss:0.45383
[915]	validation_0-mlogloss:0.453604
[916]	validation_0-mlogloss:0.45304
[917]	validation_0-mlogloss:0.452712
[918]	validation_0-mlogloss:0.452465
[919]	validation_0-mlogloss:0.452134
[920]	validation_0-mlogloss:0.452034
[921]	validation_0-mlogloss:0.451786
[922]	validation_0-mlogloss:0.451406
[923]	validation_0-mlogloss:0.450959
[924]	validation_0-mlogloss:0.450752
[925]	validation_0-mlogloss:0.450481
[926]	validation_0-mlogloss:0.450306
[927]	

...

[7195]	validation_0-mlogloss:0.103176
[7196]	validation_0-mlogloss:0.10316
[7197]	validation_0-mlogloss:0.103153
[7198]	validation_0-mlogloss:0.103154
[7199]	validation_0-mlogloss:0.103142
[7200]	validation_0-mlogloss:0.103135
[7201]	validation_0-mlogloss:0.103131
[7202]	validation_0-mlogloss:0.103127
[7203]	validation_0-mlogloss:0.103103
[7204]	validation_0-mlogloss:0.103111
[7205]	validation_0-mlogloss:0.103099
[7206]	validation_0-mlogloss:0.103096
[7207]	validation_0-mlogloss:0.103089
[7208]	validation_0-mlogloss:0.103095
[7209]	validation_0-mlogloss:0.10309
[7210]	validation_0-mlogloss:0.103095
[7211]	validation_0-mlogloss:0.103086
[7212]	validation_0-mlogloss:0.103084
[7213]	validation_0-mlogloss:0.103073
[7214]	validation_0-mlogloss:0.103069
[7215]	validation_0-mlogloss:0.103065
[7216]	validation_0-mlogloss:0.103067
[7217]	validation_0-mlogloss:0.103066
[7218]	validation_0-mlogloss:0.103044
[7219]	validation_0-mlogloss:0.103035
[7220]	validation_0-mlogloss:0.103032
[7221]	validat

[7412]	validation_0-mlogloss:0.102118
[7413]	validation_0-mlogloss:0.102124
[7414]	validation_0-mlogloss:0.102103
[7415]	validation_0-mlogloss:0.102111
[7416]	validation_0-mlogloss:0.102106
[7417]	validation_0-mlogloss:0.102107
[7418]	validation_0-mlogloss:0.102098
[7419]	validation_0-mlogloss:0.102099
[7420]	validation_0-mlogloss:0.102091
[7421]	validation_0-mlogloss:0.10208
[7422]	validation_0-mlogloss:0.102081
[7423]	validation_0-mlogloss:0.10208
[7424]	validation_0-mlogloss:0.102079
[7425]	validation_0-mlogloss:0.102065
[7426]	validation_0-mlogloss:0.102059
[7427]	validation_0-mlogloss:0.102032
[7428]	validation_0-mlogloss:0.10203
[7429]	validation_0-mlogloss:0.102025
[7430]	validation_0-mlogloss:0.102013
[7431]	validation_0-mlogloss:0.102009
[7432]	validation_0-mlogloss:0.101998
[7433]	validation_0-mlogloss:0.101994
[7434]	validation_0-mlogloss:0.101984
[7435]	validation_0-mlogloss:0.101976
[7436]	validation_0-mlogloss:0.101964
[7437]	validation_0-mlogloss:0.101963
[7438]	validati

[7629]	validation_0-mlogloss:0.101049
[7630]	validation_0-mlogloss:0.101047
[7631]	validation_0-mlogloss:0.10104
[7632]	validation_0-mlogloss:0.101038
[7633]	validation_0-mlogloss:0.101042
[7634]	validation_0-mlogloss:0.101039
[7635]	validation_0-mlogloss:0.101033
[7636]	validation_0-mlogloss:0.101023
[7637]	validation_0-mlogloss:0.101011
[7638]	validation_0-mlogloss:0.101012
[7639]	validation_0-mlogloss:0.101017
[7640]	validation_0-mlogloss:0.101011
[7641]	validation_0-mlogloss:0.101011
[7642]	validation_0-mlogloss:0.10101
[7643]	validation_0-mlogloss:0.10099
[7644]	validation_0-mlogloss:0.100992
[7645]	validation_0-mlogloss:0.100981
[7646]	validation_0-mlogloss:0.100977
[7647]	validation_0-mlogloss:0.100969
[7648]	validation_0-mlogloss:0.10097
[7649]	validation_0-mlogloss:0.100956
[7650]	validation_0-mlogloss:0.100957
[7651]	validation_0-mlogloss:0.100947
[7652]	validation_0-mlogloss:0.100945
[7653]	validation_0-mlogloss:0.100934
[7654]	validation_0-mlogloss:0.100936
[7655]	validatio

[7846]	validation_0-mlogloss:0.099987
[7847]	validation_0-mlogloss:0.099988
[7848]	validation_0-mlogloss:0.099977
[7849]	validation_0-mlogloss:0.099973
[7850]	validation_0-mlogloss:0.099975
[7851]	validation_0-mlogloss:0.099966
[7852]	validation_0-mlogloss:0.099961
[7853]	validation_0-mlogloss:0.099944
[7854]	validation_0-mlogloss:0.099937
[7855]	validation_0-mlogloss:0.099929
[7856]	validation_0-mlogloss:0.099928
[7857]	validation_0-mlogloss:0.099927
[7858]	validation_0-mlogloss:0.099922
[7859]	validation_0-mlogloss:0.09991
[7860]	validation_0-mlogloss:0.0999
[7861]	validation_0-mlogloss:0.099898
[7862]	validation_0-mlogloss:0.099896
[7863]	validation_0-mlogloss:0.099885
[7864]	validation_0-mlogloss:0.099888
[7865]	validation_0-mlogloss:0.099887
[7866]	validation_0-mlogloss:0.099879
[7867]	validation_0-mlogloss:0.099872
[7868]	validation_0-mlogloss:0.099859
[7869]	validation_0-mlogloss:0.099859
[7870]	validation_0-mlogloss:0.099857
[7871]	validation_0-mlogloss:0.099844
[7872]	validati

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.2, gamma=0, learning_rate=0.4, max_delta_step=0,
       max_depth=3, min_child_weight=2, missing=None, n_estimators=9000,
       n_jobs=1, nthread=-1, num_class=81, objective='multi:softprob',
       random_state=41, reg_alpha=0, reg_lambda=0.1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1, tree_method='auto')

In [23]:
# make predictions for validation data
y_pred = model.predict(X_test)

In [24]:
#performance of the model using validation data
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 97.68%


In [25]:
#performance of the model using validation data
#only probabilities can be used for roc_auc
# Predict class probabilities
prob_y = model.predict_proba(X_test)
# Keep only the positive class
prob_y = [p[1] for p in prob_y]
metrics.roc_auc_score(y_test, prob_y)

ValueError: multiclass format is not supported

In [26]:
#top 3 probabilities for evaluation data
preds = model.predict_proba(X_eval)
preds_idx = np.argsort(-preds, axis=1)[:,:3] #-preds for descending order
preds_idx[0]

array([27, 66, 31], dtype=int64)

In [27]:
preds_idx = np.transpose(preds_idx)
preds_idx.shape

(3, 19440)

In [28]:
preds_idx[:3]

array([[27, 27, 27, ..., 68, 68, 68],
       [66, 14, 66, ..., 31, 31,  9],
       [31, 41, 15, ...,  9,  9,  5]], dtype=int64)

In [29]:
#convert results into a dataframe
results = eval_data.iloc[:, [0,1,2,3]]
#results.rename(columns = {0: 'student_id', 1: 'level', 2: 'course', 3: 'grade'}, inplace = True)
results['major1'] = preds_idx[0]
results['major2'] = preds_idx[1]
results['major3'] = preds_idx[2]
results.head()

C:\Users\SLALITHAMBIKA\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\SLALITHAMBIKA\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,student_id,level,course,grade,major1,major2,major3
0,6Q79FVMSbTPMBhha,Senior,ECE:489,A,27,66,31
1,6Q79FVMSbTPMBhha,Sophomore,MATH:246,B,27,14,41
2,6Q79FVMSbTPMBhha,Sophomore,ECE:344,C+,27,66,15
3,6Q79FVMSbTPMBhha,Junior,ARTS:280,B,5,66,9
4,6Q79FVMSbTPMBhha,Freshman,CSI:160,AUS,21,27,45


In [32]:
#reverse encode predictions
results['major1'] = results['major1'].astype(int)
results['major1'] = labelencoder.inverse_transform(results['major1'])
results['major2'] = results['major2'].astype(int)
results['major2'] = labelencoder.inverse_transform(results['major2'])
results['major3'] = results['major3'].astype(int)
results['major3'] = labelencoder.inverse_transform(results['major3'])

In [36]:
results.tail(50)

,student_id,level,course,grade,major1,major2,major3
19390,V6eh8EGhFko9eWPk,Sophomore,ENGR:211,A+,Civil Engineering,Chemical Engineering,Electrical Engineering
19391,V6eh8EGhFko9eWPk,Sophomore,STAT:202,A+,Civil Engineering,Chemical Engineering,Electrical Engineering
19392,V6eh8EGhFko9eWPk,Junior,IE:361,B-,Industrial Engineering,Biology,Psychology
19393,V6eh8EGhFko9eWPk,Freshman,MATH:244,B+,Civil Engineering,Chemical Engineering,Electrical Engineering
19394,V6eh8EGhFko9eWPk,Senior,LEIS:106,A,Finance,Psychology,Biology
19395,V6eh8EGhFko9eWPk,Freshman,ENGR:130,B,Civil Engineering,Chemical Engineering,Electrical Engineering
19396,V6eh8EGhFko9eWPk,Junior,IE:344,A+,Industrial Engineering,Biology,Psychology
19397,V6eh8EGhFko9eWPk,Sophomore,COMM:130,D+,Psychology,Cinema,Political Science
19398,V6eh8EGhFko9eWPk,Freshman,FYP:110,S,Biology,Psychology,Civil Engineering
19399,V6eh8EGhFko9eWPk,Freshman,RHET:103,B,Biomedical Sciences,Biology,Psychology


In [35]:
#save results as a psv file
results.to_csv('results.psv', sep= '|', index = False)